#Brief description of the dataset

I am going to work with two datasets and combine them:

- Total amount of power generated through time by energy type.
- Price of power.

First we are going to load the necessary modules for this program to work:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Then, we will load the datasets that we have produced from our JS application. I need to combine different years of data:

In [ ]:
years = ['2017', '2018', '2019', '2020', '2021']

EnergyGeneration_All = pd.io.json.read_json('./energy_generation_type_date2016.json')
for x in years:
    pathEnergy = './energy_generation_type_date' + x + '.json'
    newDF = pd.io.json.read_json(pathEnergy).round()
    EnergyGeneration_All = EnergyGeneration_All.append(newDF)

EnergyGeneration_All


On the other hand we will now import the prices of energy, also on a daily basis.

In [ ]:

pathPrices = './prices.csv'

historyPrice = pd.read_csv(pathPrices, decimal=',')

historyPrice

In this case we can see that the dates are in European form, in fact they're not even interpreted as dates. Let's convert them and set them as index, so that it looks the same as the other df:

In [ ]:
historyPrice['Fecha'] = pd.to_datetime(historyPrice['Fecha'], format='%d/%m/%Y')
historyPrice.set_index('Fecha', inplace=True)
historyPrice

Now we are ready to merge this data by using the dates as an index.
This will yield only as a result the rows where the date is present for both dataframes.

In [ ]:
PriceEnergy = historyPrice.merge(EnergyGeneration_All, left_on='Fecha', right_index=True)
PriceEnergy 


#Initial plan for data exploration

In our datasets we have basically three variables: price, type of energy, and amount of energy. 
My idea is to use scatter charts, and histograms in order to have a first visual idea of the relation between these variables.

#Actions taken for data cleaning

Now that we have both dataframes merged, we can check if everything is correct.
1) In principle, there should be no NaN values. Here we get one column (Fuel + Gas, #4) with most of values NaN so we will discard it.

In [ ]:
PriceEnergy.info()
PriceEnergy.drop(columns='Fuel + Gas', inplace=True)


Now, there should be no negative values. From describe() we get some rows where the minimum is negative. 
The only place where this is relevant is in column Carbón (Coal), although the value is quite little  we will replace all those values with '0'.

In [ ]:
PriceEnergy.describe()
EnergyGeneration_All.loc[(EnergyGeneration_All['Carbón'] < 0)] = 0

Moreover, the maximum of each column must be lower than maximum of Total Generation. 
Here is can be seen that the maximum generation is 943229, which should be higher than the maximum of the rest of values for any given day. Which is true.


In [ ]:
PriceEnergy.max()

#Exploratory data analysis

I am going to draw several plots in order to understand the data that we have.

If we plot the total generated energy as a whole, I realise that there is a lot of noise around this indicator, so I am going to work with a moving average of 14 days.
In the illustration below it can be seen the real value vs the moving average. Note that the real values have such a high fluctuation, that it is even hard to tell the value of the function.

In [ ]:
PriceEnergy['Average'] = pd.Series.rolling(PriceEnergy['Generación total'], 14).mean().round()
PriceEnergy

fig, ax = plt.subplots()

ax.plot(PriceEnergy['Generación total'])
ax.plot(PriceEnergy['Average'])

plt.show()

From the plot above it is not easy to tell if there has been major changes in energy consumption in Spain.
We will stack the series by eliminating the year in order to analyze this in a more visual manner.

In [ ]:
generationHistory = pd.DataFrame(PriceEnergy['Average'])

generationHistory['año'] = generationHistory.index.to_series().dt.year
generationHistory['fecha'] = generationHistory.index.to_series().dt.strftime('%m-%d')
generationHistory.set_index(['año', 'fecha'], inplace=True)
generationHistory = generationHistory.unstack(level=0)
#generationHistory['avg_16_20'] = df.loc[2017:]

older_years = generationHistory['Average'].loc[:, '2016':'2020'].mean(axis=1)
older_years


Now we are ready to plot it.

In [ ]:
plt.plot(generationHistory)
plt.show()

Another column we may use is the amount of energy that is renewable vs non renewable:

In [ ]:
PriceEnergy['Renovable'] = PriceEnergy['Hidráulica'] + PriceEnergy['Hidroeólica'] + PriceEnergy['Eólica'] + PriceEnergy['Solar fotovoltaica'] + PriceEnergy['Solar térmica'] + PriceEnergy['Otras renovables'] + PriceEnergy['Residuos renovables']

PriceEnergy['No renovable'] = PriceEnergy['Generación total'] - PriceEnergy['Renovable'] 

PriceEnergy['Renovable_media'] = pd.Series.rolling(PriceEnergy['Renovable'], 14).mean().round()
PriceEnergy.to_csv('out.csv')
PriceEnergy

Now I would like to have a look at the data combined.

After several rounds of testing, I have decided to define a function so that we can draw multiple plots quickly. The function will show:

- The distribution of the column that I am analyzing (in the first attempt, the Total Generation)
- Price and Energy vs Date.
- Scatter of Price vs Energy. 

When we plot the data (price and energy) of all 5 years, it turns out that the distribution "looks pretty normal". 
But at first it is hard to tell any correlation between them. In fact, by late 2021 we see that energy remains constant while price skyrockets. In fact we get a Pearson correlation of 0.12.

In [ ]:
def plot_PriceEnergy_Energyhist(myCol, dateFrom, dateTo):
    PriceEnergy_select = PriceEnergy.loc[dateFrom:dateTo]
    
    #First plot: Energy histogram (to answer the question, is it normal distribution?)
    sns.distplot(PriceEnergy_select[myCol], bins=10)
    plt.show()
    
    sns.distplot(PriceEnergy_select['Precio'], bins=10)
    plt.show()
    fig, axs = plt.subplots(2)

    #Second plot: Price and energy generation vs. Date
    color = 'tab:red' 
    axs[0].set_xlabel('Date')
    axs[0].set_ylabel('precio')
    axs[0].plot(PriceEnergy_select['Precio'], color=color)

    axs0_right = axs[0].twinx()
    axs0_right.set_ylabel('Energy')
    axs0_right.plot(PriceEnergy_select[myCol])

    #Third plot: 
    axs[1].scatter(PriceEnergy_select[myCol], PriceEnergy_select['Precio'])
    plt.show()

    
    
    print("Correlación Pearson: ", PriceEnergy_select['Precio'].corr(PriceEnergy_select[myCol], method='spearman'))
   

plot_PriceEnergy_Energyhist('Average', '2016-01-01', '2021-12-31')

But what happens when we limit the amount of data analyzed to the last year:

In [ ]:
plot_PriceEnergy_Energyhist('Average', '2021-01-01', '2021-12-31')

Again it does not look too promising.
Now let's focus at another hypothesis: more expensive energies need to come into play because the renewable energies (the cheapest of all) can't cope with demand.
Consequently, if we limit the analysis to renewables, we should see a negative correlation of some sort.

In [ ]:
plot_PriceEnergy_Energyhist('Renovable_media', '2016-01-01', '2021-12-31')

If we pay close attention to the plot, for example zooming into (2016-01-01 until 2017-07-31) we could tell some correspondence between the renewable energy increasing and the price decreasing.
If we assume a normal distribution, the linear coefficient raises to -0.62.

This suggests that there is a correlation between the renewable energy that is generated and the price.  

In [ ]:
plot_PriceEnergy_Energyhist('Renovable_media', '2016-01-01', '2017-07-31')

Moreover, the fact that by reducing the time frame this is more evident, suggests that we shouldn't compare data from different years, because there may be other macroeconomic factors out of our control affecting the price. Therefore, we will do the analysis separately  will focus the analysis in the Renewable energy but in separate years.

By executing the function separately for each year, we get the following Pearson's coefficients:
- 2016: -.823
- 2017: -.011
- 2018: -.795
- 2019: +.11
- 2020: .074
- 2021: -.63

As we can see, it is rather variable. Some years show a strong correlation, while others a weak one. This suggests that the wind blowing less has some influence, but it is not the only one in a complex process such as the energy price setting.




In [ ]:
plot_PriceEnergy_Energyhist('Renovable_media', '2021-01-01', '2021-12-31')

#Key findings and insights

There seems to be some degree of inverse correlation between the power of renewable energies generated at a given day, and the price of those energies.
This makes sense, because renewable energies are cheap to produce. However they are unstable. Wind does not blow whenever we want. So when this happens, other less cheap energies come into play.
Such correlation is sometimes stronger, sometimes weaker, because there are other factors that need to be taken into account (such as CO2 rights).

Based on these findings, I would suggest to assess the Cost / Benefit of installing more renewable energy plants. The question would be, whether the taxpayer money invested in more windmills would compensate for the reduced energy bill.
With the caution that there may be other factors coming into place.

#Formal significance test

Hypothesis: the energy prices have gone this year up until 150 EUR. We would be interested in knowing if this is going to be a long-term trend (H0) or if it is going to be a short-term variation (H1).
For this we choose a p-value of 0.05. Meaning that if at least 5% of the daily prices is above 150 EUR, then we can accept this is a permanent trend.

From the CDF for prices calculated above, the probability of the price going above 150 EUR, this year, is of 6 days out of 259 days, that is 0.023%.
As this result is below the p-level, then we must reject the null hypothesis and conclude that this is a short term trend.

#Conclusions

At this point, it can be said that the price of energy shows a correlation which has been dominant over the past few years, although this is not the only one.
It can be said with an X percent confidence that an increase of renewable power in the country can lead to a decrease in Y euros in the MWh.

However there are other possible factors intervening in th
In order to be able to analyze them, we would need additional data: CO2 markets.